In [11]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
sys.path.append('../')

sys.path.append('../../REA_with_CESM2/ensembles')
sys.path.append('../../REA_with_CESM2/data_extraction')
from ensemble_GKLT import ensemble_GKLT
from data_extractor import extract_rea,extract_initial

from experiment_configuration.experiment import experiment

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
'''
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--experiment", type=str)
parser.add_argument("--verbose", action='store_true')
parser.add_argument("--overwrite", action='store_true')
parser.add_argument("--realm", type=str)
parser.add_argument("--h_identifier", type=str)
parser.add_argument("--var_name", type=str)
command_line_arguments = parser.parse_args()
'''

'\nimport argparse\nparser = argparse.ArgumentParser()\nparser.add_argument("--experiment", type=str)\nparser.add_argument("--verbose", action=\'store_true\')\nparser.add_argument("--overwrite", action=\'store_true\')\nparser.add_argument("--realm", type=str)\nparser.add_argument("--h_identifier", type=str)\nparser.add_argument("--var_name", type=str)\ncommand_line_arguments = parser.parse_args()\n'

In [23]:
variable = 'Z500'
realm = 'atm'
h_identifier = 'h1'
overwrite = False

In [8]:
for experiment_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    print(experiment_identifier)
    # load experiment configuration settings
    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)

    extract_rea(
        exp,
        variable = variable,
        realm = realm,
        h_identifier = h_identifier,
        overwrite = overwrite,
    )

    if '1' in experiment_identifier:
        extract_initial(
            exp,
            variable = variable,
            realm = realm,
            h_identifier = h_identifier,
            overwrite = overwrite,
        )
    



c1
c2
c3


In [9]:
extract_rea(
    exp,
    variable = 'Z500',
    realm = 'atm',
    h_identifier = 'h1',
    overwrite = False,
)

In [12]:
extract_initial(
    exp,
    variable = 'Z500',
    realm = 'atm',
    h_identifier = 'h1',
    overwrite = False,
)

In [ ]:
for experiment_identifier in ["c1","p1"]:
    print(experiment_identifier)
    # load configuration settings
    experiment_configuration = importlib.import_module(f"experiment_configuration.{experiment_identifier}")

    # finalize experiment configuration
    from experiment_configuration.experiment import experiment
    exp = experiment(experiment_configuration.config)

    naming_d = {
        "project": 'REA_output',
        "product": exp.product_name,
        "institute": 'NCAR',
        "model": 'CESM2',
        "experiment": f"{exp.initial_conditions_name}-initial",
        "time_frequency": "day",
        "realm": realm_dict[realm],
        "variable": variable_dict[variable],
    }

    

    for i,initial_condition in enumerate(exp.initial_conditions):
        ens_name = f"ens{str(i+1).zfill(3)}"
        out_dir = '/'.join([exp.dir_work] + [v for k,v in naming_d.items()] + [ens_name])
        case_identifier = initial_condition.split('.')[-4] + '_' + initial_condition.split('/')[-1].split('-')[0]
        out_file_name = f"{out_dir}/{variable_dict[variable]}_day_CESM2_{naming_d['experiment']}_{ens_name}_{exp.initial_condition_fake_year}.nc"
        if os.path.isfile(out_file_name) == False or overwrite:
            archive_fldr = f"{exp.dir_archive}/GKLT/initial_{exp.initial_conditions_name}/{case_identifier}"
            h_files = glob.glob(f"{archive_fldr}/{realm}/hist/*{h_identifier}*.nc")

            try:
                with xr.open_mfdataset(h_files) as nc:
                    x = nc[variable]
                    initial_condition_year = x.time.dt.year.values[0]
                    fake_year = 2025
                    x = x.assign_coords(time=np.array([cftime.DatetimeNoLeap(exp.initial_condition_fake_year,month,day) for month,day in zip(x.time.dt.month.values,x.time.dt.day.values)]))
                    x = x.assign_coords(sim=case_identifier)
                    ds = xr.Dataset({variable_dict[variable]:x})
                    ds.attrs = nc.attrs
                    ds.attrs['initial_condition'] = initial_condition
                    ds.attrs['initial_condition_year'] = initial_condition_year
                    ds.attrs['compset'] = exp.compset
                    ds.attrs['readme'] = "https://github.com/peterpeterp/REA_heat_wEU_JJA"
                    ds['time'].attrs['comment'] = f'This simulation represents the climate state of {exp.initial_conditions_name}. The year in the time axis can be ignored.'
                    os.makedirs(out_dir, exist_ok=True)
                    ds.to_netcdf(out_file_name)
            except:
                pass

c1
p1
